In [10]:
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

root_dir = '/home/thien/research/pseudogroups/CivilComments/splitpgl_sweep_logs/all_wd0.01_lr1e-05/'
meta_data_path = '/home/thien/research/datasets/jigsaw/data/all_data_with_identities.csv'
part = 'part1'
seed = 0
epoch = 0
val_or_test = 'val'


file_path = root_dir + f'{part}_s{seed}/output_{val_or_test}_epoch_{epoch}.csv'
output_df = pd.read_csv(file_path)

metadata_df = pd.read_csv(meta_data_path)
test_df = metadata_df.iloc[output_df[f'indices_None_epoch_{epoch}_val']]
placeholder = (test_df['toxicity'] >= 0.5).astype(int)
test_df['labels'] = placeholder
test_df.reset_index(inplace=True)

/home/thien/anaconda3/envs/spuriouscorr/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
all_groups = [
            'male',
            'female',
            'christian',
            'muslim',
            'other_religion',
            'black',
            'white',
            'LGBTQ'
    ]

pred_col_name = f'y_pred_None_epoch_{epoch}_val'
true_col_name = f'y_true_None_epoch_{epoch}_val'

group_acc_dict = {}
group_n_dict = {}

for toxic in range(2):  # in 0 or 1
    for g in range(len(all_groups)):
        group_idx = toxic*len(all_groups) + g
        idxs = (test_df['labels'] == toxic) & (test_df[all_groups[g]] == 1)
        total_n_g = sum(idxs)
        group_n_dict[(all_groups[g],toxic)] = total_n_g
        if total_n_g <= 0:
            group_acc_dict[(all_groups[g],toxic)] = 1  # vacuously perfect
            continue
        correct_pred = sum(test_df[idxs]['labels'] == output_df[idxs][pred_col_name])
        group_acc_dict[(all_groups[g],toxic)] =  correct_pred/total_n_g
wg_acc = min(v for k, v in group_acc_dict.items())
avg_acc = sum(output_df[pred_col_name] == output_df[true_col_name])/len(output_df)

In [12]:
wg_acc

0.3853211009174312

In [13]:
avg_acc

0.9296148738379814